In [1]:
!pip install langchain langchain_community langchain-text-splitter langchain-chroma

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter


In [2]:
!pip install langchain-groq  # Install the langchain_groq package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.3 MB/s eta 0:00:00


In [6]:
!pip install langchain-community

  Using cached langchain_community-0.3.17-py3-none-any.whl.metadata (2.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [37]:
import pandas as pd
import numpy as np
import langchain
import langchain_community
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma


In [38]:
df=pd.read_csv('/content/Finance_data.csv')
df_fin=df.to_dict(orient='records')

In [39]:
for entry in df_fin:
  prompt = f"I'm a {entry['age']}-year-old {entry['gender']} looking to invest in {entry['Avenue']} for {entry['Purpose']} over the next {entry['Duration']}. What are my options?"
  print(prompt)

I'm a 34-year-old Female looking to invest in Mutual Fund for Wealth Creation over the next 1-3 years. What are my options?
I'm a 23-year-old Female looking to invest in Mutual Fund for Wealth Creation over the next More than 5 years. What are my options?
I'm a 30-year-old Male looking to invest in Equity for Wealth Creation over the next 3-5 years. What are my options?
I'm a 22-year-old Male looking to invest in Equity for Wealth Creation over the next Less than 1 year. What are my options?
I'm a 24-year-old Female looking to invest in Equity for Wealth Creation over the next Less than 1 year. What are my options?
I'm a 24-year-old Female looking to invest in Mutual Fund for Wealth Creation over the next 1-3 years. What are my options?
I'm a 27-year-old Female looking to invest in Equity for Wealth Creation over the next 3-5 years. What are my options?
I'm a 21-year-old Male looking to invest in Mutual Fund for Wealth Creation over the next 3-5 years. What are my options?
I'm a 35-yea

#
Pre-Processng the Data into Prompt-Response Format

In [40]:
# Convert the data to prompt-response format
prompt_response_data = []
for entry in df_fin:
    prompt = f"I'm a {entry['age']}-year-old {entry['gender']} looking to invest in {entry['Avenue']} for {entry['Purpose']} over the next {entry['Duration']}. What are my options?"
    response = (
        f"Based on your preferences, here are your investment options:\n"
        f"- Mutual Funds: {entry['Mutual_Funds']}\n"
        f"- Equity Market: {entry['Equity_Market']}\n"
        f"- Debentures: {entry['Debentures']}\n"
        f"- Government Bonds: {entry['Government_Bonds']}\n"
        f"- Fixed Deposits: {entry['Fixed_Deposits']}\n"
        f"- PPF: {entry['PPF']}\n"
        f"- Gold: {entry['Gold']}\n"
        f"Factors considered: {entry['Factor']}\n"
        f"Objective: {entry['Objective']}\n"
        f"Expected returns: {entry['Expect']}\n"
        f"Investment monitoring: {entry['Invest_Monitor']}\n"
        f"Reasons for choices:\n"
        f"- Equity: {entry['Reason_Equity']}\n"
        f"- Mutual Funds: {entry['Reason_Mutual']}\n"
        f"- Bonds: {entry['Reason_Bonds']}\n"
        f"- Fixed Deposits: {entry['Reason_FD']}\n"
        f"Source of information: {entry['Source']}\n"
    )
    prompt_response_data.append({"prompt": prompt, "response": response})

prompt_response_data[:5]

[{'prompt': "I'm a 34-year-old Female looking to invest in Mutual Fund for Wealth Creation over the next 1-3 years. What are my options?",
  'response': 'Based on your preferences, here are your investment options:\n- Mutual Funds: 1\n- Equity Market: 2\n- Debentures: 5\n- Government Bonds: 3\n- Fixed Deposits: 7\n- PPF: 6\n- Gold: 4\nFactors considered: Returns\nObjective: Capital Appreciation\nExpected returns: 20%-30%\nInvestment monitoring: Monthly\nReasons for choices:\n- Equity: Capital Appreciation\n- Mutual Funds: Better Returns\n- Bonds: Safe Investment\n- Fixed Deposits: Fixed Returns\nSource of information: Newspapers and Magazines\n'},
 {'prompt': "I'm a 23-year-old Female looking to invest in Mutual Fund for Wealth Creation over the next More than 5 years. What are my options?",
  'response': 'Based on your preferences, here are your investment options:\n- Mutual Funds: 4\n- Equity Market: 3\n- Debentures: 2\n- Government Bonds: 1\n- Fixed Deposits: 5\n- PPF: 6\n- Gold: 7\

# Storing data in VEctor DB

In [41]:
from langchain.docstore.document import Document
documents = []
for entry in prompt_response_data:
    combined_text = f"Prompt: {entry['prompt']}\nResponse: {entry['response']}"
    documents.append(Document(page_content=combined_text))


In [42]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
texts = text_splitter.split_documents(documents)

In [43]:
from langchain.embeddings import HuggingFaceEmbeddings

# Define the Hugging Face embeddings
hg_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [44]:
from langchain.vectorstores import Chroma

# Directory to persist the vector database
persist_directory = 'docs/chroma/'

# Define the batch size based on the error message (less than 166)
BATCH_SIZE = 150  # Slightly below the max limit for safety

# Function to create Chroma vector database in batches
def create_chroma_vector_db(documents, embedding, persist_directory, batch_size=BATCH_SIZE):
    # Initialize an empty Chroma database
    vectordb_fin = None

    # Process documents in batches
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]

        if vectordb_fin is None:
            # Create Chroma DB with the first batch
            vectordb_fin = Chroma.from_documents(
                documents=batch,
                embedding=embedding,
                persist_directory=persist_directory
            )
        else:
            # Add subsequent batches to the existing database
            vectordb_fin.add_documents(batch)

    return vectordb_fin

# Call the function with the provided documents and embeddings
vectordb_fin = create_chroma_vector_db(documents=texts, embedding=hg_embeddings, persist_directory=persist_directory)


In [45]:
from google.colab import userdata
# LLM with function call
from langchain_groq import ChatGroq
import os
groq_api_key=userdata.get('groq_api_key')
os.environ["GROQ_API_KEY"]=groq_api_key

In [50]:
llm=ChatGroq(groq_api_key=groq_api_key,model_name='llama-3.3-70b-versatile')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7d9eca957890>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7d9eca726450>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [51]:
from langchain.chains import RetrievalQA
retriever_fin = vectordb_fin.as_retriever(search_kwargs={"k":5})
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever_fin, return_source_documents=False)
query = "I am a 21 year college student, what scheme is better for me?"
result = qa({"query": query})
result

{'query': 'I am a 21 year college student, what scheme is better for me?',
 'result': "As a 21-year-old college student looking to invest in a mutual fund for a period of less than 1 year, I would recommend considering liquid funds or ultra-short-term debt funds. \n\nThese types of funds typically invest in low-risk, short-term debt instruments and provide easy liquidity, making them suitable for short-term goals. They usually offer relatively stable returns and are less volatile compared to other types of mutual funds.\n\nHowever, please keep in mind that mutual fund investments always carry some level of risk, and it's essential to assess your personal risk tolerance and financial goals before investing. It's also a good idea to consult with a financial advisor or conduct your own research before making any investment decisions."}